In [1]:
import pandas as pd
import numpy as np
import itertools
import pandas_gbq
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT
*
FROM `perceptive-ivy-290216.f1_api.race_lap_time`  A
# WHERE A.Year=2024
# AND A.GP="Monaco Grand Prix"
# AND A.DRIVER='HAM'
ORDER BY LapNumber, LapStartTime
"""
track3=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

Downloading: 100%|██████████|


In [4]:
track2=track3[(track3["GP"]=='Belgian Grand Prix')&(track3["Year"]==2024)]

In [5]:
track2.head()
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]

In [6]:
track2['LapTime']= pd.to_timedelta(track2["LapTime"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/4173085553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track2['LapTime']= pd.to_timedelta(track2["LapTime"])


In [7]:
# track2['LapTime'].dt.total_seconds().min()*1.07
track2=track2[track2['LapNumber']!=1.0]
track2.loc[:, "LapTime (s)"] = track2["LapTime"].dt.total_seconds()

In [8]:
quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
quicklaps.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime (s)
20477,0 days 00:58:57.253000,LEC,16,0 days 00:01:50.240000,2.0,1.0,NaT,NaT,0 days 00:00:31.831000,0 days 00:00:48.675000,0 days 00:00:29.734000,0 days 00:57:38.844000,0 days 00:58:27.519000,0 days 00:58:57.253000,303.0,203.0,215.0,NaN,True,MEDIUM,2.0,True,Ferrari,0 days 00:57:07.013000,NaT,1,1.0,False,,False,True,2024,Belgian Grand Prix,110.240
20480,0 days 00:58:57.650000,HAM,44,0 days 00:01:49.849000,2.0,1.0,NaT,NaT,0 days 00:00:31.284000,0 days 00:00:49.211000,0 days 00:00:29.354000,0 days 00:57:39.019000,0 days 00:58:28.230000,0 days 00:58:57.584000,329.0,202.0,220.0,307.0,True,MEDIUM,2.0,True,Mercedes,0 days 00:57:07.801000,NaT,1,2.0,False,,False,True,2024,Belgian Grand Prix,109.849
20485,0 days 00:58:58.780000,PER,11,0 days 00:01:50.240000,2.0,1.0,NaT,NaT,0 days 00:00:31.365000,0 days 00:00:49.207000,0 days 00:00:29.668000,0 days 00:57:39.872000,0 days 00:58:29.079000,0 days 00:58:58.747000,325.0,202.0,217.0,308.0,True,MEDIUM,2.0,True,Red Bull Racing,0 days 00:57:08.540000,NaT,1,3.0,False,,False,True,2024,Belgian Grand Prix,110.240
20491,0 days 00:58:59.513000,PIA,81,0 days 00:01:50.164000,2.0,1.0,NaT,NaT,0 days 00:00:31.056000,0 days 00:00:49.480000,0 days 00:00:29.628000,0 days 00:57:40.397000,0 days 00:58:29.877000,0 days 00:58:59.505000,335.0,201.0,216.0,313.0,True,MEDIUM,2.0,True,McLaren,0 days 00:57:09.349000,NaT,1,4.0,False,,False,True,2024,Belgian Grand Prix,110.164
20494,0 days 00:59:00.024000,RUS,63,0 days 00:01:50.160000,2.0,1.0,NaT,NaT,0 days 00:00:31.075000,0 days 00:00:49.700000,0 days 00:00:29.385000,0 days 00:57:40.899000,0 days 00:58:30.599000,0 days 00:58:59.984000,328.0,199.0,220.0,312.0,True,MEDIUM,2.0,True,Mercedes,0 days 00:57:09.864000,NaT,1,5.0,False,,False,True,2024,Belgian Grand Prix,110.160


In [9]:
transformed_laps = quicklaps.copy()
transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

# order the team from the fastest (lowest median lap time) to slower
team_order = (
    transformed_laps[["Team", "LapTime (s)"]].groupby("Team").median()["LapTime (s)"].sort_values().index
)
print(team_order)

Index(['Mercedes', 'McLaren', 'Ferrari', 'Red Bull Racing', 'Williams', 'RB',
       'Alpine', 'Aston Martin', 'Kick Sauber', 'Haas F1 Team'],
      dtype='object', name='Team')


In [10]:
transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('median')
transformed_laps.tail()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime (s),median
437887,0 days 02:16:18.772000,MAG,20,0 days 00:01:47.848000,44.0,2.0,NaT,NaT,0 days 00:00:30.979000,0 days 00:00:48.058000,0 days 00:00:28.811000,0 days 02:15:01.882000,0 days 02:15:49.940000,0 days 02:16:18.751000,317.0,204.0,221.0,317.0,True,HARD,27.0,True,Haas F1 Team,0 days 02:14:30.924000,NaT,1,15.0,False,,False,True,2024,Belgian Grand Prix,107.848,109.634
437891,0 days 02:16:22.779000,BOT,77,0 days 00:01:47.516000,44.0,3.0,NaT,NaT,0 days 00:00:31.022000,0 days 00:00:47.699000,0 days 00:00:28.795000,0 days 02:15:06.241000,0 days 02:15:53.940000,0 days 02:16:22.735000,316.0,208.0,222.0,313.0,False,MEDIUM,9.0,True,Kick Sauber,0 days 02:14:35.263000,NaT,1,16.0,False,,False,True,2024,Belgian Grand Prix,107.516,109.591
437893,0 days 02:16:28.878000,TSU,22,0 days 00:01:47.969000,44.0,2.0,NaT,NaT,0 days 00:00:30.937000,0 days 00:00:48.057000,0 days 00:00:28.975000,0 days 02:15:11.802000,0 days 02:15:59.859000,0 days 02:16:28.834000,NaN,208.0,221.0,313.0,True,HARD,30.0,False,RB,0 days 02:14:40.909000,NaT,1,17.0,False,,False,True,2024,Belgian Grand Prix,107.969,109.363
437899,0 days 02:16:38.198000,SAR,2,0 days 00:01:48.121000,44.0,3.0,NaT,NaT,0 days 00:00:31.021000,0 days 00:00:48.123000,0 days 00:00:28.977000,0 days 02:15:21.087000,0 days 02:16:09.210000,0 days 02:16:38.187000,323.0,202.0,220.0,313.0,False,HARD,20.0,True,Williams,0 days 02:14:50.077000,NaT,1,18.0,False,,False,True,2024,Belgian Grand Prix,108.121,109.153
437902,0 days 02:16:40.974000,HUL,27,0 days 00:01:48.954000,44.0,3.0,NaT,NaT,0 days 00:00:31.003000,0 days 00:00:48.902000,0 days 00:00:29.049000,0 days 02:15:22.991000,0 days 02:16:11.893000,0 days 02:16:40.942000,314.0,199.0,219.0,NaN,True,MEDIUM,24.0,True,Haas F1 Team,0 days 02:14:52.020000,NaT,1,19.0,False,,False,True,2024,Belgian Grand Prix,108.954,109.634


In [11]:
fig_pace=px.box(
    transformed_laps.sort_values(by=["median","LapNumber"]),
    x="Team",
    y="LapTime (s)",
    color='Team',
    template="xgridoff",
    title="<b>Team Pace Plot for the {} {}</b>".format(year,gp),
    # palette=team_palette,
    # whiskerprops=dict(color="white"),
    # boxprops=dict(edgecolor="white"),
    # medianprops=dict(color="grey"),
    # capprops=dict(color="white"),
    height=700, width=1200,

color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
)
fig_pace.update_traces(opacity=1)
fig_pace.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    margin=dict(l=60, r=5, t=50, b=60),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_pace.show()

In [15]:
fig_pace.write_html("{} {} Team Pace Plot.html".format(year,gp),full_html=False, include_plotlyjs='cdn')

In [12]:
#Get MINIMUM OF THE MEDIAN LAP TIMES

fastest_lap = transformed_laps["median"].min()
transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
transformed_laps_relative_median=transformed_laps[["Team","Delta","Delta_Percent"]].drop_duplicates()
transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
transformed_laps_relative_median

,Team,Delta,Delta_Percent
20480,Mercedes,0.0000,100.0%
20491,McLaren,0.1725,100.16%
20477,Ferrari,0.2655,100.246%
20485,Red Bull Racing,0.3400,100.315%
20503,Williams,1.1880,101.1%
20507,RB,1.3980,101.295%
20504,Alpine,1.4880,101.378%
20501,Aston Martin,1.5795,101.463%
20506,Kick Sauber,1.6260,101.506%
20509,Haas F1 Team,1.6690,101.546%


In [13]:
for index, row in transformed_laps_relative_median.iterrows():
  if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
    transformed_laps_relative_median.loc[index, 'Delta']=0.001
transformed_laps_relative_median

,Team,Delta,Delta_Percent
20480,Mercedes,0.0010,100.0%
20491,McLaren,0.1725,100.16%
20477,Ferrari,0.2655,100.246%
20485,Red Bull Racing,0.3400,100.315%
20503,Williams,1.1880,101.1%
20507,RB,1.3980,101.295%
20504,Alpine,1.4880,101.378%
20501,Aston Martin,1.5795,101.463%
20506,Kick Sauber,1.6260,101.506%
20509,Haas F1 Team,1.6690,101.546%


In [26]:
fig_median=px.bar(
    transformed_laps_relative_median,
    x="Team",
    y="Delta",
    color='Team',
    template="xgridoff",
    title="<b>Team Pace (Median Lap Times) for the {} {}</b>".format(year,gp),
    text="Delta_Percent",
    height=700, width=1200,
color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
)

fig_median.update_traces(textposition='outside')

fig_median.update_traces(opacity=0.85)

fig_median.update_traces( marker_line_color='white',marker_line_width=1.5)

fig_median.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig_median.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig_median.update_yaxes(range=[transformed_laps_relative_median['Delta'].max()+0.6, 0])

# fig['layout']['yaxis']['autorange'] = "reversed"

fig_median.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig_median.update_xaxes(title_text='')      # Remove axis titles
fig_median.update_yaxes(title_text='')

fig_median.update_xaxes(
        title_standoff = 75
)

fig_median.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_median.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

fig_median.show()

In [19]:
fig_median.write_html("{} {} Team Median Plot.html".format(year,gp),full_html=False, include_plotlyjs='cdn')

In [15]:
#Get MINIMUM OF ALL LAP TIMES
transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('min')

fastest_lap_all = transformed_laps["min_all"].min()
transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
transformed_laps_relative_min_all=transformed_laps[["Team","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
transformed_laps_relative_min_all

,Team,Delta_Min_All,Delta_Percent_All
20485,Red Bull Racing,0.000,100.0%
20491,McLaren,0.862,100.823%
20477,Ferrari,1.663,101.588%
20480,Mercedes,1.952,101.864%
20504,Alpine,2.256,102.155%
20506,Kick Sauber,2.318,102.214%
20507,RB,2.734,102.611%
20503,Williams,2.789,102.664%
20509,Haas F1 Team,3.147,103.006%
20501,Aston Martin,3.350,103.2%


In [16]:
for index, row in transformed_laps_relative_min_all.iterrows():
  if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
    transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
transformed_laps_relative_min_all

,Team,Delta_Min_All,Delta_Percent_All
20485,Red Bull Racing,0.001,100.0%
20491,McLaren,0.862,100.823%
20477,Ferrari,1.663,101.588%
20480,Mercedes,1.952,101.864%
20504,Alpine,2.256,102.155%
20506,Kick Sauber,2.318,102.214%
20507,RB,2.734,102.611%
20503,Williams,2.789,102.664%
20509,Haas F1 Team,3.147,103.006%
20501,Aston Martin,3.350,103.2%


In [27]:
fig_min=px.bar(
    transformed_laps_relative_min_all,
    x="Team",
    y="Delta_Min_All",
    color='Team',
    template="xgridoff",
    title="<b>Team Pace (Fastest Lap Times) for the {} {}</b>".format(year,gp),
    text="Delta_Percent_All",
    height=700, width=1200,
color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
)

fig_min.update_traces(textposition='outside')

fig_min.update_traces(opacity=0.85)

fig_min.update_traces( marker_line_color='white',marker_line_width=1.5)

fig_min.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig_min.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig_min.update_yaxes(range=[transformed_laps_relative_min_all['Delta_Min_All'].max()+0.5, 0])

# fig['layout']['yaxis']['autorange'] = "reversed"

fig_min.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig_min.update_xaxes(title_text='')      # Remove axis titles
fig_min.update_yaxes(title_text='')

fig_min.update_xaxes(
        title_standoff = 75
)

fig_min.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_min.update_yaxes(ticksuffix = "  ")
fig_min.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

fig_min.show()

In [25]:
fig_min.write_html("{} {} Team Fastest Lap Plot.html".format(year,gp),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi GP Plot

In [32]:
#2024
race_multi=["Bahrain Grand Prix", "Saudi Arabian Grand Prix", "Australian Grand Prix", "Japanese Grand Prix","Chinese Grand Prix","Miami Grand Prix","Emilia Romagna Grand Prix","Monaco Grand Prix","Canadian Grand Prix","Spanish Grand Prix","Austrian Grand Prix","British Grand Prix","Hungarian Grand Prix","Belgian Grand Prix"]
#2023
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
# race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2024
for i in race_multi:
    track2=track3[(track3["GP"]==i)&(track3["Year"]==year_multi)]
    year=track2['Year'].iloc[0]
    gp=track2['GP'].iloc[0]
    track2['LapTime']= pd.to_timedelta(track2["LapTime"])
    track2=track2[track2['LapNumber']!=1.0]
    quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
    transformed_laps = quicklaps.copy()
    transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

    # order the team from the fastest (lowest median lap time) tp slower
    team_order = (
        transformed_laps[["Team", "LapTime (s)"]].groupby("Team").median()["LapTime (s)"].sort_values().index
    )
    print(team_order)
    transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('median')
    transformed_laps.tail()
    fig_pace=px.box(
        transformed_laps.sort_values(by=["median","LapNumber"]),
        x="Team",
        y="LapTime (s)",
        color='Team',
        template="xgridoff",
        title="<b>Team Pace Plot for the {} {}</b>".format(year_multi,i),
        height=700, width=1200,

    color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
            }
    )
    fig_pace.update_traces(opacity=1)
    fig_pace.update_layout(
        title_x=0.5,
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow",
        margin=dict(l=50, r=5, t=35, b=50)
    )
    fig_pace.write_html("{} {} Team Pace Plot.html".format(year_multi,i),full_html=False, include_plotlyjs='cdn')
    #Get MINIMUM OF THE MEDIAN LAP TIMES

    fastest_lap = transformed_laps["median"].min()
    transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
    transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
    transformed_laps_relative_median=transformed_laps[["Team","Delta","Delta_Percent"]].drop_duplicates()
    transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
    transformed_laps_relative_median
    for index, row in transformed_laps_relative_median.iterrows():
        if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
         transformed_laps_relative_median.loc[index, 'Delta']=0.001
    transformed_laps_relative_median
    fig_median=px.bar(
        transformed_laps_relative_median,
        x="Team",
        y="Delta",
        color='Team',
        template="xgridoff",
        title="<b>Team Pace (Median Lap Times) for the {} {}</b>".format(year_multi,i),
        text="Delta_Percent",
        height=700, 
        width=1200,
        color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
            }
    )

    fig_median.update_traces(textposition='outside')

    fig_median.update_traces(opacity=0.85)

    fig_median.update_traces( marker_line_color='white',marker_line_width=1.5)

    fig_median.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_median.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_median.update_yaxes(range=[transformed_laps_relative_median['Delta'].max()+0.6, 0])

    fig_median.update_layout(
        xaxis={'side': 'top'},
        yaxis={'side': 'left'}
    )

    fig_median.update_xaxes(title_text='')      # Remove axis titles
    fig_median.update_yaxes(title_text='')

    fig_median.update_xaxes(
            title_standoff = 75
    )

    fig_median.update_layout(
        title_x=0.5,
        margin=dict(l=60, r=5, t=110, b=30),
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=12,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_median.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
    fig_median.write_html("{} {} Team Median Plot.html".format(year_multi,i),full_html=False, include_plotlyjs='cdn')
  #Get MINIMUM OF ALL LAP TIMES
    transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform('min')
    fastest_lap_all = transformed_laps["min_all"].min()
    transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
    transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
    transformed_laps_relative_min_all=transformed_laps[["Team","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
    transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
    transformed_laps_relative_min_all
    for index, row in transformed_laps_relative_min_all.iterrows():
        if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
            transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
    fig_min=px.bar(
        transformed_laps_relative_min_all,
        x="Team",
        y="Delta_Min_All",
        color='Team',
        template="xgridoff",
        title="<b>Team Pace (Fastest Lap Times) for the {} {}</b>".format(year,gp),
        text="Delta_Percent_All",
        height=700, width=1200,
    color_discrete_map={
                     "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                    }
    )

    fig_min.update_traces(textposition='outside')

    fig_min.update_traces(opacity=0.85)

    fig_min.update_traces( marker_line_color='white',marker_line_width=1.5)

    fig_min.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_min.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_min.update_yaxes(range=[transformed_laps_relative_min_all['Delta_Min_All'].max()+0.5, 0])

    # fig['layout']['yaxis']['autorange'] = "reversed"

    fig_min.update_layout(
        xaxis={'side': 'top'},
        yaxis={'side': 'left'}
    )

    fig_min.update_xaxes(title_text='')      # Remove axis titles
    fig_min.update_yaxes(title_text='')

    fig_min.update_xaxes(
            title_standoff = 75
    )

    fig_min.update_layout(
        title_x=0.5,
        margin=dict(l=60, r=5, t=110, b=30),
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_min.update_yaxes(ticksuffix = "  ")
    fig_min.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
    fig_min.write_html("{} {} Team Fastest Lap Plot.html".format(year_multi,i),full_html=False, include_plotlyjs='cdn')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['Red Bull Racing', 'Ferrari', 'McLaren', 'Mercedes', 'Haas F1 Team',
       'Aston Martin', 'Kick Sauber', 'Alpine', 'RB', 'Williams'],
      dtype='object', name='Team')
Index(['Red Bull Racing', 'Ferrari', 'McLaren', 'Mercedes', 'Aston Martin',
       'Haas F1 Team', 'Kick Sauber', 'Alpine', 'Williams', 'RB'],
      dtype='object', name='Team')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['Ferrari', 'McLaren', 'Red Bull Racing', 'Mercedes', 'Aston Martin',
       'Haas F1 Team', 'Alpine', 'RB', 'Kick Sauber', 'Williams'],
      dtype='object', name='Team')
Index(['Red Bull Racing', 'McLaren', 'Mercedes', 'Ferrari', 'Aston Martin',
       'RB', 'Williams', 'Kick Sauber', 'Haas F1 Team', 'Alpine'],
      dtype='object', name='Team')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

...

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

Index(['Red Bull Racing', 'Mercedes', 'Williams', 'Ferrari', 'McLaren', 'RB',
       'Alpine', 'Kick Sauber', 'Aston Martin'],
      dtype='object', name='Team')
Index(['Red Bull Racing', 'Mercedes', 'McLaren', 'Aston Martin',
       'Haas F1 Team', 'RB', 'Alpine', 'Kick Sauber', 'Ferrari', 'Williams'],
      dtype='object', name='Team')
Index(['Red Bull Racing', 'McLaren', 'Ferrari', 'Mercedes', 'Alpine',
       'Haas F1 Team', 'RB', 'Aston Martin', 'Kick Sauber', 'Williams'],
      dtype='object', name='Team')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['McLaren', 'Ferrari', 'Mercedes', 'Red Bull Racing', 'Haas F1 Team',
       'Alpine', 'RB', 'Aston Martin', 'Kick Sauber', 'Williams'],
      dtype='object', name='Team')
Index(['McLaren', 'Mercedes', 'Red Bull Racing', 'Ferrari', 'Aston Martin',
       'Haas F1 Team', 'Williams', 'RB', 'Kick Sauber', 'Alpine'],
      dtype='object', name='Team')
Index(['Mercedes', 'Red Bull Racing', 'McLaren', 'Ferrari', 'Aston Martin',
       'RB', 'Haas F1 Team', 'Williams', 'Kick Sauber', 'Alpine'],
      dtype='object', name='Team')
Index(['Mercedes', 'McLaren', 'Ferrari', 'Red Bull Racing', 'Williams', 'RB',
       'Alpine', 'Aston Martin', 'Kick Sauber', 'Haas F1 Team'],
      dtype='object', name='Team')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3579/1109977575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [253]:
# #Get AVERAGE OF ALL LAP TIMES
# transformed_laps['avg_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Team']).transform(np.mean)

# fastest_lap_avg = transformed_laps["avg_all"].min()
# transformed_laps['Delta_Avg_All']= transformed_laps["avg_all"] - (fastest_lap_avg)
# transformed_laps['Delta_Percent_Avg']= (transformed_laps["avg_all"]/(fastest_lap_avg)-1)*100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].round(3)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].astype(str)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+"%"
# transformed_laps_relative_avg_all=transformed_laps[["Team","Delta_Avg_All","Delta_Percent_Avg"]].drop_duplicates()
# transformed_laps_relative_avg_all=transformed_laps_relative_avg_all.sort_values("Delta_Avg_All")
# transformed_laps_relative_avg_all

In [254]:
# for index, row in transformed_laps_relative_avg_all.iterrows():
#   if transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']==0.0:
#     transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']=0.001
# transformed_laps_relative_avg_all

In [255]:
# fig=px.bar(
#     transformed_laps_relative_avg_all,
#     x="Team",
#     y="Delta_Avg_All",
#     color='Team',
#     template="xgridoff",
#     title="<b>Team Pace (Average Lap Times) for the {} {}</b>".format(year,gp),
#     text="Delta_Percent_Avg",
#     height=700, width=1200,
# color_discrete_map={
#                  "Alpine": "#0093cc",
#                  "Aston Martin": "#229971",
#                  "Ferrari": "#E80020",
#                  "Haas F1 Team": "#B6BABD",
#                  "Kick Sauber": "#52e252",
#                  "McLaren": "#FF8000",
#                  "Mercedes": "#27F4D2",
#                  "RB": "#6692FF",
#                  "Red Bull Racing": "#3671C6",
#                  "Williams": "#64C4FF" ,
#                  "Alfa Romeo":"#C92D4B",
#                  "AlphaTauri":"#5E8FAA",
#                  "Racing Point":"#F596C8",
#                  "Renault":"#FFF500",
#                  "Toro Rosso":"#469bff",
#                  "Force India":"#F596C8",
#                  "Sauber":"#9B0000"
#                  }
# )

# fig.update_traces(textposition='outside')

# fig.update_traces(opacity=0.85)

# fig.update_traces( marker_line_color='white',marker_line_width=1.5)

# fig.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_yaxes(range=[2, 0])

# # fig['layout']['yaxis']['autorange'] = "reversed"

# fig.update_layout(
#     xaxis={'side': 'top'},
#     yaxis={'side': 'left'}
# )

# fig.update_xaxes(title_text='')      # Remove axis titles
# fig.update_yaxes(title_text='')

# fig.update_xaxes(
#         title_standoff = 75
# )

# fig.update_layout(
#     title_x=0.5,
#     margin=dict(l=60, r=5, t=110, b=30),
#     hoverlabel=dict(
#     bgcolor="white",
#     font_size=16,
#     font_family="PT Sans Narrow"
#     ),
#     yaxis = dict(tickfont = dict(size=15)),
#     xaxis = dict(tickfont = dict(size=15)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=14,
#         color="Black"
#     ),
#     title_font_family="PT Sans Narrow"
# )
# fig.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

# fig.show()

In [256]:
# fig.write_html("{} {} Team Avg Lap Plot.html".format(year,gp))